In [1]:
from core.adam import Adam


In [2]:
import pandas as pd
import numpy as np
from core.checker import parameters_checker
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from core.utils.utils import measurement_inverter, scaler, calculate_likelihood, calculate_entropy, calculate_multistep_loss
from numpy.linalg import eigvals
import nlopt


In [3]:
from core.estimator import estimator, selector
from core.creator import creator, initialiser, architector, filler
from core.utils.ic import ic_function

from smooth.adam_general._adam_general import adam_fitter, adam_forecaster

import warnings

In [28]:
# Generate random monthly time series data
np.random.seed(41)  # For reproducibility
n_points = 24  # 2 years of monthly data
time_series = np.random.randint(1, 100, size=n_points).cumsum()  # Random walk with strictly positive integers
dates = pd.date_range(start='2023-01-01', periods=n_points, freq='M')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series}, index=dates)


In [23]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [34]:
%%R -i ts_df
library(smooth)

model <- adam(ts_df, model = "ANN", lags = c(12))

forecast(model, h = 12)
model$B

   alpha    level 
 1.00000 64.99598 


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:56: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


In [13]:
model = "ANN"
lags = [12]

multisteps = False,
lb = None,
ub = None,
maxtime = None,
print_level = 1, # 1 or 0
maxeval = None,
h = 12


# Assume that the model is not provided
# these will be default arguments
profiles_recent_provided = False
profiles_recent_table = None


In [31]:
ts_df

,value
2023-01-31,65
2023-02-28,101
2023-03-31,114
2023-04-30,213
2023-05-31,294
2023-06-30,360
2023-07-31,450
2023-08-31,474
2023-09-30,560
2023-10-31,587


In [30]:
adam = Adam(model, lags)
adam.fit(ts_df, h = h)
fc = adam.predict()
fc['forecast']

Initial value is not selected. Switching to optimal.
Initial parameters: [1.000e-01 1.574e+02]


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/filtheo/smooth/python/smooth/adam_general/core/utils/utils.py:329: RuntimeWarning: overflow encountered in square
  return np.sqrt(np.sum(errors**2) / obs_in_sample)
/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


<class 'core.estimator.OptimizeResult'>


/home/filtheo/smooth/.venv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


2025-01-31    13.475472
2025-02-28    13.475472
2025-03-31    13.475472
2025-04-30    13.475472
2025-05-31    13.475472
2025-06-30    13.475472
2025-07-31    13.475472
2025-08-31    13.475472
2025-09-30    13.475472
2025-10-31    13.475472
2025-11-30    13.475472
2025-12-31    13.475472
Freq: M, dtype: float64

In [33]:
adam.adam_estimated

{'B': array([1.000e-01, 1.574e+02]),
 'CF_value': 1611.9508210967872,
 'n_param_estimated': 2,
 'log_lik_adam_value': {'value': -1611.9508210967872, 'nobs': 24, 'df': 3},
 'arima_polynomials': None}

In [32]:
fc['initial_estimated']

[True]